In [1]:
import numpy as np
import pandas as pd
import math



In [2]:
class TreeNode: # Tree_Node
    
    def __init__(self,data,output):
        
        self.data=data     #feature_name on which it split
        self.children={}
        self.index=-1
        self.output=output # majority class value
        
    def add_child(self,feature_val,obj):
        
        self.children[feature_val]=obj
         

In [35]:
class DecisionTreeClasssifier:
    
    def __init__(self):
        self._root=None  # root of tree
        
    def __count__unique(self,Y):
        d={}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1;
        return d
    
    def __entropy(self,Y):
        
        freq=self.__count__unique(Y)
        tot=len(Y)
        sum=0
        for i in freq:
            sum+=((-freq[i]/tot)*math.log2(freq[i]/tot))
        return sum
    
    def __gain_ratio(self,X,Y,selected_feature):
       
        orig_info=self.__entropy(Y)
        info_after_split=0
        value=set(X[:,selected_feature])
        df=pd.DataFrame(X)
        df[df.shape[1]]=Y
        tot=df.shape[0]
        split_info=0
        for i in value:
            df1=df[df[selected_feature]==i]
            current_size=df1.shape[0]
            info_after_split+=(current_size/tot)*self.__entropy(df1[df1.shape[1]-1])
            split_info+=(-current_size/tot)*math.log2((current_size/tot))
         
        if(split_info==0):
            return math.inf
        info_gain=orig_info-info_after_split
        gain_ratio=info_gain/split_info
        
        return gain_ratio
    
    def __gini_(self,Y):
        
        freq=self.__count__unique(Y)
        tot=len(Y)
        sum=1
        for i in freq:
            sum-=(freq[i]/tot)**2
        return sum
    
    def __gini_ratio(self,X,Y,selected_feature):
       
        orig_info=self.__gini_(Y)
        info_after_split=0
        value=set(X[:,selected_feature])
        df=pd.DataFrame(X)
        df[df.shape[1]]=Y
        tot=df.shape[0]
        
        for i in value:
            df1=df[df[selected_feature]==i]
            current_size=df1.shape[0]
            info_after_split+=(current_size/tot)*self.__gini_(df1[df1.shape[1]-1])
            
      
        info_gain=orig_info-info_after_split
      
        
        return info_gain
    
    def __decisionTree(self,X,Y,features,level,metric,classes):
        
        if(len(set(Y))==1 ):  # BASE condition 1
            print("level",level)
            output=None
            
            for i in classes:

                if(i not in Y):
                    print("Count of",i,"=0")
                else:
                    output=i
                    print("Count of",i,len(Y))

    
    
            if(metric=="gain_ratio"):
                print("Current Entropy with gain_ratio",0)
            elif(metric=="gini_ratio"):
                print("Current Entropy with gini_ratio",0)

            print("leaf Node Reached")
            return TreeNode(None,output)
        
         
        if(len(features)==0):  # BASE condition 2
            print("level",level)
            output=None
            
            freq=self.__count__unique(Y)
            max_count=-math.inf
            for i in classes:
                if(i not in freq):
                    print("Count of",i,"=0")
                else:
                    print("Count of",i,freq[i])
                    if(max_count<freq[i]):
                        max_count=freq[i]
                        output=i

            if(metric=="gain_ratio"):
                print("Current Entropy with gain_ratio",self.__entropy(Y))
            elif(metric=="gini_ratio"):
                print("Current Entropy with gini_ratio",self.__gini_(Y))

            print("leaf Node Reached")
            return TreeNode(None,output)
        
        
        
        
        max_gain=-math.inf
        final_feature=None
      
        for i in features:   # check which feature Give maximum Gain
            if(metric=="gain_ratio"):
                current_gain=self.__gain_ratio(X,Y,i)
            if(metric=="gini_ratio"):
                current_gain=self.__gini_ratio(X,Y,i)
                
            if(current_gain>max_gain):
                max_gain=current_gain
                final_feature=i
                
        print("level",level)
        
        freq=self.__count__unique(Y)
        output=None
        max_count=-math.inf
        
        for i in classes:
            if(i not in freq):
                print("Count of",i,"=0")
            else:
                if(freq[i]>max_count):
                    output=i
                    max_count=freq[i]
                print("Count of",i,freq[i])  
                
    
        if metric == "gain_ratio" :        
            print("Current Entropy is =",self.__entropy(Y))
            print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
            print()
        elif metric == "gini_index":
            print("Current Gini Index is =",self.__gini_index(Y))
            print("Splitting on feature  X[",final_feature,"] with gini gain ",max_gain,sep="")
            print()
            
        
        current_node=TreeNode(final_feature,output)
        
        unique_val=set(X[:,final_feature])
        df=pd.DataFrame(X)
        df[df.shape[1]]=Y
        
        idx=features.index(final_feature)
        features.remove(final_feature)
         
        for i in unique_val:  # make children 
            
            df1=df[df[final_feature]==i]
            node= self.__decisionTree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.add_child(i,node)
            
        features.insert(idx,final_feature)
        
        return current_node
    
        
        
        
    def fit(self,X,Y,metric="gain_ratio"):
        
        features= [i for i in range(len(X[0]))]
        classes =set(Y)
        level=0
         
        self._root=self.__decisionTree(X,Y,features,level,metric,classes)
    
    def predict_for(self,data,node):
        
        if(len(node.children)==0):
            return node.output
        val=data[node.data]
        if(val not in node.children):
            return node.output
        
        return self.predict_for(data,node.children[val])
    
    def predict(self,X):
        
        Y=np.array([0 for i in range(len(X))])
        for i in range(len(X)):
             Y[i] = self.predict_for(X[i],self._root)
        return Y
   
    
    def score(self,X,Y):
        Y_pred=self.predict(X)
        cont=0;
        for i in range(len(Y_pred)):
            if(Y_pred[i]==Y[i]):
                cont+=1
        return cont/len(Y_pred)
    
    

In [29]:
clf = DecisionTreeClasssifier()
x=np.array([[0,0],[0,1],[1,0],[1,1]])
y=np.array([0,1,1,1])
clf.fit(x,y)
clf.predict(x)
clf.score(x,y)

level 0
Count of 0 1
Count of 1 3
Current Entropy is = 0.8112781244591328
Splitting on feature  X[0] with gain ratio 0.31127812445913283

level 1
Count of 0 1
Count of 1 1
Current Entropy is = 1.0
Splitting on feature  X[1] with gain ratio 1.0

level 2
Count of 0 1
Count of 1 =0
Current Entropy with gain_ratio 0
leaf Node Reached
level 2
Count of 0 =0
Count of 1 1
Current Entropy with gain_ratio 0
leaf Node Reached
level 1
Count of 0 =0
Count of 1 2
Current Entropy with gain_ratio 0
leaf Node Reached


1.0

In [39]:
from sklearn import datasets
# Generating a random dataset
X,Y = datasets.make_classification(n_samples=100, n_features=5,random_state=0)
# To reduce the values a feature can take ,converting floats to int
for i in range(len(X)):
    for j in range(len(X[0])):
        X[i][j] = int(X[i][j])

clg2=DecisionTreeClasssifier()
clg2.fit(X,Y,metric="gini_ratio")
clg2.score(X,Y)

level 0
Count of 0 50
Count of 1 50
level 1
Count of 0 24
Count of 1 27
level 2
Count of 0 13
Count of 1 21
level 3
Count of 0 6
Count of 1 17
level 4
Count of 0 5
Count of 1 13
level 5
Count of 0 5
Count of 1 13
Current Entropy with gini_ratio 0.4012345679012346
leaf Node Reached
level 4
Count of 0 =0
Count of 1 2
Current Entropy with gini_ratio 0
leaf Node Reached
level 4
Count of 0 =0
Count of 1 1
Current Entropy with gini_ratio 0
leaf Node Reached
level 4
Count of 0 =0
Count of 1 1
Current Entropy with gini_ratio 0
leaf Node Reached
level 4
Count of 0 1
Count of 1 =0
Current Entropy with gini_ratio 0
leaf Node Reached
level 3
Count of 0 2
Count of 1 3
level 4
Count of 0 1
Count of 1 1
level 5
Count of 0 1
Count of 1 =0
Current Entropy with gini_ratio 0
leaf Node Reached
level 5
Count of 0 =0
Count of 1 1
Current Entropy with gini_ratio 0
leaf Node Reached
level 4
Count of 0 =0
Count of 1 1
Current Entropy with gini_ratio 0
leaf Node Reached
level 4
Count of 0 1
Count of 1 1
level 5

0.92

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
clg1=DecisionTreeClassifier()
clg1.fit(X,Y)
clg1.score(X,Y)

0.92